In [10]:
import requests
import json
import sqlite3

## Definition function

In [2]:
def get_buienradar_data():
    url = "https://json.buienradar.nl/"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # raises exception for HTTP errors

        data = response.json()  # convert JSON to Python dict
        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching Buienradar data: {e}")
        return None


In [3]:

# Example usage
if __name__ == "__main__":
    buienradar_data = get_buienradar_data()
    if buienradar_data:
        # print general weather info
        actual = buienradar_data.get("actual", {})
        stationmeasurements = actual.get("stationmeasurements", [])

        print("Number of weather stations:", len(stationmeasurements))
        if stationmeasurements:
            first_station = stationmeasurements[0]
            print(f"First station: {first_station.get('stationname')}")
            print(f"Temperature: {first_station.get('temperature')} °C")

Number of weather stations: 40
First station: Meetstation Arcen
Temperature: 14.4 °C


In [25]:
# Step 2: Create SQLite database and table
def setup_database(db_name="buienradar_weather.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Enable foreign key support (must be set per connection)
    cursor.execute("PRAGMA foreign_keys = ON;")

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather_stations (
            stationid INTEGER PRIMARY KEY,
            stationname TEXT,
            lat REAL,
            lon REAL,
            regio TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather_station_measurements (
            measurementid INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp DATETIME,
            temperature REAL,
            groundtemperature REAL,
            feeltemperature REAL,
            windgusts REAL,
            windspeedBft REAL,
            humidity REAL,
            precipitation REAL,
            sunpower REAL,
            stationid INTEGER,
            FOREIGN KEY (stationid) REFERENCES weather_stations (stationid)
                ON UPDATE CASCADE
                ON DELETE CASCADE
        )
    """)

    conn.commit()
    return conn

In [31]:
print("🗄️ Setting up database...")
conn = setup_database()
conn

🗄️ Setting up database...


## Testing Zone

In [27]:
url = "https://json.buienradar.nl/"
response= requests.get(url, timeout=10)
data = response.json()

stations = data.get("actual", {}).get("stationmeasurements", [])
stations[0]


{'$id': '4',
 'stationid': 6391,
 'stationname': 'Meetstation Arcen',
 'lat': 51.5,
 'lon': 6.2,
 'regio': 'Venlo',
 'timestamp': '2025-10-22T13:40:00',
 'weatherdescription': 'Zwaar bewolkt',
 'iconurl': 'https://cdn.buienradar.nl/resources/images/icons/weather/30x30/c.png',
 'fullIconUrl': 'https://cdn.buienradar.nl/resources/images/icons/weather/96x96/C.png',
 'graphUrl': 'https://www.buienradar.nl/nederland/weerbericht/weergrafieken/c',
 'winddirection': 'ZW',
 'temperature': 14.6,
 'groundtemperature': 14.7,
 'feeltemperature': 14.6,
 'windgusts': 2.6,
 'windspeed': 1.4,
 'windspeedBft': 1,
 'humidity': 79.0,
 'precipitation': 0.0,
 'sunpower': 214.0,
 'rainFallLast24Hour': 0.2,
 'rainFallLastHour': 0.0,
 'winddirectiondegrees': 233}

In [28]:
cursor = conn.cursor()
cursor.execute("""
    INSERT INTO weather_stations (stationid, stationname, lat, lon, regio)
    VALUES (?, ?, ?, ?, ?)
""", (
    stations[0].get("stationid"),
    stations[0].get("stationname"),
    stations[0].get("lat"),
    stations[0].get("lon"),
    stations[0].get("regio")
    )
)
conn.commit()

In [30]:
cursor = conn.cursor()
cursor.execute("""
    INSERT INTO weather_station_measurements ( timestamp, temperature, groundtemperature, feeltemperature, windgusts, windspeedBft, humidity, precipitation, sunpower, stationid)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", (
    stations[0].get("timestamp"),
    stations[0].get("temperature"),
    stations[0].get("groundtemperature"),
    stations[0].get("feeltemperature"),
    stations[0].get("windgusts"),
    stations[0].get("humidity"),
    stations[0].get("windspeedBft"),
    stations[0].get("precipitation"),
    stations[0].get("sunpower"),
    stations[0].get("stationid")
    )
)
conn.commit()

In [ ]:
def insert_station_data(conn, data):
    cursor = conn.cursor()

    # Enable foreign keys
    cursor.execute("PRAGMA foreign_keys = ON;")

    # Get list of stations from JSON
    stations = data.get("actual", {}).get("stationmeasurements", [])

    # Keep track of existing station IDs to avoid duplicates
    # (Optional: you can skip this if you always clear the tables first)
    cursor.execute("SELECT stationname, stationid FROM weather_stations")
    existing_stations = {row[0]: row[1] for row in cursor.fetchall()}

    for s in stations:
        stationname = s.get("stationname")
        lat = s.get("lat")
        lon = s.get("lon")
        regio = s.get("regio") if "regio" in s else None  # optional field

        # Insert station only if it doesn't exist
        if stationname in existing_stations:
            stationid = existing_stations[stationname]
        else:
            cursor.execute("""
                INSERT INTO weather_stations (stationname, lat, lon, regio)
                VALUES (?, ?, ?, ?)
            """, (stationname, lat, lon, regio))
            stationid = cursor.lastrowid
            existing_stations[stationname] = stationid

        # Insert measurement
        cursor.execute("""
            INSERT INTO weather_station_measurements
            (temperature, groundtemperature, feeltemperature, windgusts, windspeedBft,
             humidity, precipitation, sunpower, stationid)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            s.get("temperature"),
            s.get("groundtemperature"),
            s.get("feeltemperature"),
            s.get("windgusts"),
            s.get("windspeedBft"),
            s.get("humidity"),
            s.get("precipitation"),
            s.get("sunpower"),
            stationid
        ))

    # Commit everything in a single transaction
    conn.commit()
    print(f"✅ Inserted {len(stations)} station measurements into the database.")

